### Normalizing Genres

In [1]:
import pandas as pd
df = pd.read_csv('Data/title_basics_filtered.csv.gz')
df.head(2)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama


In [2]:
df = df.drop(columns=['originalTitle', 'isAdult', 'titleType'])

In [3]:
## create a col with a list of genres
df['genres_split'] = df['genres'].str.split(',')
df

,tconst,primaryTitle,startYear,endYear,runtimeMinutes,genres,genres_split
0,tt0035423,Kate & Leopold,2001.0,NaN,118,"Comedy,Fantasy,Romance","[Comedy, Fantasy, Romance]"
1,tt0062336,The Tango of the Widower and Its Distorting Mi...,2020.0,NaN,70,Drama,[Drama]
2,tt0068865,Lives of Performers,2016.0,NaN,90,Drama,[Drama]
3,tt0069049,The Other Side of the Wind,2018.0,NaN,122,Drama,[Drama]
4,tt0088751,The Naked Monster,2005.0,NaN,100,"Comedy,Horror,Sci-Fi","[Comedy, Horror, Sci-Fi]"
...,...,...,...,...,...,...,...
81884,tt9914942,Life Without Sara Amat,2019.0,NaN,74,Drama,[Drama]
81885,tt9915872,The Last White Witch,2019.0,NaN,97,"Comedy,Drama,Fantasy","[Comedy, Drama, Fantasy]"
81886,tt9916170,The Rehearsal,2019.0,NaN,51,Drama,[Drama]
81887,tt9916190,Safeguard,2020.0,NaN,95,"Action,Adventure,Thriller","[Action, Adventure, Thriller]"


In [4]:
genres_split = df['genres'].str.split(',')

exploded_genres = df.explode('genres_split')
exploded_genres


unique_genres = genres_split.explode().unique()
unique_genres

array(['Comedy', 'Fantasy', 'Romance', 'Drama', 'Horror', 'Sci-Fi',
       'Biography', 'Mystery', 'Musical', 'Action', 'Adventure', 'Crime',
       'Thriller', 'Music', 'Animation', 'Family', 'History', 'War',
       'Sport', 'Western', 'Adult', 'Reality-TV', 'News', 'Talk-Show',
       'Game-Show'], dtype=object)

In [5]:
unique_genres = sorted(exploded_genres['genres_split'].unique())

In [6]:
title_genres = exploded_genres[['tconst', 'genres_split']].copy()
title_genres.head()

,tconst,genres_split
0,tt0035423,Comedy
0,tt0035423,Fantasy
0,tt0035423,Romance
1,tt0062336,Drama
2,tt0068865,Drama


In [7]:
# Making the genre mapper dictionary
genre_ints = range(len(unique_genres))
genre_map = dict(zip(unique_genres, genre_ints))
genre_map

{'Action': 0,
 'Adult': 1,
 'Adventure': 2,
 'Animation': 3,
 'Biography': 4,
 'Comedy': 5,
 'Crime': 6,
 'Drama': 7,
 'Family': 8,
 'Fantasy': 9,
 'Game-Show': 10,
 'History': 11,
 'Horror': 12,
 'Music': 13,
 'Musical': 14,
 'Mystery': 15,
 'News': 16,
 'Reality-TV': 17,
 'Romance': 18,
 'Sci-Fi': 19,
 'Sport': 20,
 'Talk-Show': 21,
 'Thriller': 22,
 'War': 23,
 'Western': 24}

In [8]:
genre_id_map = dict(zip(unique_genres, range(len(unique_genres))))
genre_id_map

{'Action': 0,
 'Adult': 1,
 'Adventure': 2,
 'Animation': 3,
 'Biography': 4,
 'Comedy': 5,
 'Crime': 6,
 'Drama': 7,
 'Family': 8,
 'Fantasy': 9,
 'Game-Show': 10,
 'History': 11,
 'Horror': 12,
 'Music': 13,
 'Musical': 14,
 'Mystery': 15,
 'News': 16,
 'Reality-TV': 17,
 'Romance': 18,
 'Sci-Fi': 19,
 'Sport': 20,
 'Talk-Show': 21,
 'Thriller': 22,
 'War': 23,
 'Western': 24}

In [9]:
# Define a function to map genre lists to genre IDs
def map_genre_list(genre_list):
    return [genre_map.get(item, -1) for item in genre_list]

# Apply the custom function to the 'genres_split' column
df['genre_id'] = df['genres_split'].apply(map_genre_list)

# Drop the 'genres_split' column
df = df.drop(columns='genres_split')

In [10]:
genre_lookup = pd.DataFrame({'Genre_Name': genre_id_map.keys(),
                            'Genre_ID': genre_id_map.values()})
genre_lookup.head()

,Genre_Name,Genre_ID
0,Action,0
1,Adult,1
2,Adventure,2
3,Animation,3
4,Biography,4


### Saving the MySQL tables with tconst as the primary key

1. Creating a datatype schema for to_sql
   - Check the dtypes of your DataFrame: (df.dtypes)
       - create a dictionary with each column name as a key and the corresponding SQL datatype as the value
   - Find the corresponding SQLalchemy datatype to use in the table
   
   - Note on the string columns:
       - for columns with 'object' dtype, you should use:
           - the string type if the column will become your primary key
           - otherwise, use the text data type
           - Both String/Text will need the maximum string length to use for that column

In [11]:
# check the dtypes
df.dtypes

tconst             object
primaryTitle       object
startYear         float64
endYear           float64
runtimeMinutes      int64
genres             object
genre_id           object
dtype: object

In [12]:
## Example
from sqlalchemy.types import *
## Calculate max string lengths for object columns
key_len = df['tconst'].fillna('').map(len).max()
title_len = df['primaryTitle'].fillna('').map(len).max()
## Create a schema dictonary using Sqlalchemy datatype objects
df_schema = {
    "tconst": String(key_len+1), 
    "primaryTitle": Text(title_len+1),
    'startYear':Float(),
    'endYear':Float(),
    'runtimeMinutes':Integer()}